# Data Cleaning - full data set

In [1]:
%run setup.ipynb

c:\Users\olive\anaconda3\envs\base_data_analysis\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
# import modin.pandas as pd

In [3]:
from utils import get_station_data, merge_values_counts

# Load

In [4]:
import pyproj
pyproj.datadir.get_data_dir()

'C:\\Users\\olive\\anaconda3\\envs\\base_data_analysis\\Library\\share\\proj'

In [5]:
# city boundaries of washington
washington_boundary = gpd.read_file(config['raw_data_paths']['washington_outline'])
washington_boundary = washington_boundary.drop(columns=[
    'AREAMILES', 'OBJECTID', 'STATE_CITY', 'CAPITAL', 'WEB_URL', 'GLOBALID', 'CREATOR', 'CREATED', 'EDITOR','EDITED', 'SHAPEAREA', 'SHAPELEN'
    ])
washington_boundary.head()

c:\Users\olive\anaconda3\envs\base_data_analysis\Lib\site-packages\pyogrio\core.py:36: RuntimeWarning: Could not detect PROJ data files. Set PROJ_LIB environment variable to the correct path.
  _init_proj_data()


CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)

In [ ]:
# initialized rides data 
df = pd.read_parquet(config['processed_data_paths']['rides_init'])
df.info()

# Features
- is_within_city: ride is within the city boundaries, i.e. Start and End are both within the boundaries
- is_holiday: Day of ride is a holiday
- ride_duration: ride duration in s

## Ride within boundaries of city center
- Are the rides within the city center service area? The activities outside might not be that interesting
- Problem: no map of the full service boundaries is available!

In [ ]:
# NOTE gpd thinks in longitude and latitude, not the other way around
def check_if_ride_within_city_boundaries(df, city_boundary) -> pd.Series:
    geometry = [LineString([(lng1, lat1), (lng2, lat2)]) for lat1, lng1, lat2, lng2 in zip(df.start_lat, df.start_lng, df.end_lat, df.end_lng)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    rides_with_city_info = gpd.sjoin(gdf, city_boundary, how="left", predicate="intersects", lsuffix='point', rsuffix='city')
    return rides_with_city_info.index_city.notna()

def check_if_start_within_city_boundaries(df, city_boundary) -> pd.Series:
    geometry = [Point((lng1, lat1)) for lat1, lng1 in zip(df.start_lat, df.start_lng)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    rides_with_city_info = gpd.sjoin(gdf, city_boundary, how="left", predicate="within", lsuffix='point', rsuffix='city')
    return rides_with_city_info.index_city.notna()

def check_if_end_within_city_boundaries(df, city_boundary) -> pd.Series:
    geometry = [Point(lng2, lat2) for lat2, lng2 in zip(df.end_lat, df.end_lng)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    rides_with_city_info = gpd.sjoin(gdf, city_boundary, how="left", predicate="within", lsuffix='point', rsuffix='city')
    return rides_with_city_info.index_city.notna()

df['start_is_within_city']  = check_if_start_within_city_boundaries(df[['start_lat', 'start_lng', 'end_lat', 'end_lng']], washington_boundary)
df['end_is_within_city']    = check_if_end_within_city_boundaries(df[['start_lat', 'start_lng', 'end_lat', 'end_lng']], washington_boundary)
df['ride_is_within_city']   = df['start_is_within_city'] & df['end_is_within_city']

In [ ]:
df.head()

In [ ]:
# TODO why this here? why is a new index created? 
# It does not matter, so the new index can be taken
if 'index' in df.columns: #hasattr(df, 'index'):
    df = df.drop(columns='index')

In [ ]:
# TODO move to other ipynb
mask_2023 = df.started_at.dt.year >= 2023

print('rides that start outside the city limits: ', sum(~df.loc[mask_2023, 'start_is_within_city']) / df[mask_2023].shape[0])
print('rides that end outside the city limits: ',   sum(~df.loc[mask_2023, 'end_is_within_city']) / df[mask_2023].shape[0])

In [ ]:
# TODO plot the rides that are outside the city boundaries

## Holiday

In [ ]:
df['is_holiday'] = df.started_at.dt.date.apply(lambda x: x in US_HOLIDAYS.date)

In [ ]:
# mask_4thjuly = df.started_at.dt.date == datetime.date(2023, 7, 4)
# df[mask_4thjuly]

## Ride Duration

In [ ]:
df['ride_duration'] = df.ended_at - df.started_at
df['ride_duration'] = df['ride_duration'].apply(lambda x: x.seconds)

# Cleaning
- separate redundant information and create separate tables
  - station table: station_id, lat, lng (lat, log as mean values), start, end
- transform data
  - rideable_type -> 0,1,2,... OR classic: True, False
  - member_casual -> 0,1,2,...
- data integrity
  - lat, lng in Washington bbox
  - if start / end - lat / lng == null, then remove
- reduce data type to reasonable precision

## Ride duration

### Drop rides that are too short to be true
Only keep rides that lasted longer than 30s.

In [ ]:
# ride duration below 30s
rides_too_short_to_be_true = df.ride_duration < 30

In [ ]:
rides_too_short_to_be_true.sum() / df.shape[0]

In [ ]:
df = df[~rides_too_short_to_be_true]

### Tag rides that are too long to be true
Ride durations of > 1d are exceptional and will be tagged for the following analysis.
Reasons for long ride duration
- Rides have rented a bike for multiple days on purpose.
- Riders must have made an error when handing back the bike.

In [ ]:
# tag rides with ride duration > 12h

In [ ]:
# LOF for outlier detection

## Latitude and Longitude

Check if cooordinates in bounding box

In [ ]:
# service area bbox
min_lng, min_lat = -77.454987, 38.701588
max_lng, max_lat = -76.784821, 39.168400
service_area_bbox = (min_lng, min_lat, max_lng, max_lat)

In [ ]:
mask_lat_lng_outside_bbox = ((df.start_lng < min_lng) | (df.start_lng > max_lng) | (df.end_lng < min_lng) | (df.end_lng > max_lng) |
(df.start_lat < min_lat) | (df.start_lat > max_lat) | (df.end_lat < min_lat) | (df.end_lat > max_lat))

mask_lat_lng_outside_bbox.sum()

In [ ]:
df = df[~mask_lat_lng_outside_bbox]

## Stations

Clean station data such that theres only one lat, lng per station, in order to **???**
- simplify the data: Replace the bike location with station location if bike is returned at a station

1. Get all station data from rides table
2. Build station table
3. Clean main data set with station table

Questions
- Are stations moved?

Problems:
- stations have many locations that vary strongly
  - Can the station locations be determined by the available data set?
- station_id, station_name have more unique values than station_id
  - What does that mean?
- Are stations moved? What happens in the data set?
- Errors in the bike location up to 10m

Assumptions
- a station's location is determined by where most of the bikes are put.
- all ride data associated with 

Steps in order to build station table
1. clean station names
2. set precision for lat and long
3. remove station coordinate outliers
4. calculate averages of location coordinates

### Clean Station Names

In [ ]:
# clean station names
# TODO is there a way to make it more efficient? 
df.start_station_name = df.start_station_name.str.strip()
df.end_station_name = df.end_station_name.str.strip()

### Some overall stuff

In [ ]:
mask_start = ~df.start_station_id.isna()
mask_end = ~df.end_station_id.isna()

### Use docked bikes

In [ ]:
mask_docked_bikes = df.rideable_type=='docked_bike'

stations_docked_bikes = [None, None]
stations_docked_bikes[0] = get_station_data(df, mask_docked_bikes & mask_start, ['ended_at', 'start_station_name', 'start_station_id', 'start_lat', 'start_lng'], NDEC_LAT_LNG, False)
stations_docked_bikes[1] = get_station_data(df, mask_docked_bikes & mask_end, ['ended_at', 'end_station_name', 'end_station_id', 'end_lat', 'end_lng'], NDEC_LAT_LNG, False)

stations_docked_bikes = pd.concat(stations_docked_bikes, axis=0)

stations_docked_bikes = stations_docked_bikes.groupby(['station_name', 'station_id']).agg({
    'lat' : ['median', 'mean', 'std', 'count'],
    'lng' : ['median', 'mean', 'std', 'count'],
    'date': ['min', 'max']
    })

stations_docked_bikes.columns = ['_'.join(col) for col in stations_docked_bikes.columns]
stations_docked_bikes = stations_docked_bikes.reset_index()
stations_docked_bikes.head()

In [ ]:
stations_serviced = pd.concat(
    [df[['ended_at', 'start_station_id', 'start_station_name']].rename(columns=lambda x: x.replace('start_','')), 
     df[['ended_at', 'end_station_id', 'end_station_name']].rename(columns=lambda x: x.replace('end_',''))],
    axis=0
    )
stations_serviced.ended_at = stations_serviced.ended_at.dt.date
stations_serviced = stations_serviced.groupby(['station_id', 'station_name']).agg({'ended_at':['min', 'max']})
stations_serviced.columns = ['_'.join(col) for col in stations_serviced.columns]
stations_serviced = stations_serviced.reset_index()
stations_serviced

In [ ]:
check_stations_services = pd.merge(stations_docked_bikes, stations_serviced, on=['station_id', 'station_name'], how='left')
check_stations_services['check'] = check_stations_services.date_max - check_stations_services.ended_at_max
check_stations_services

In [ ]:
# get station where data is too old to be true
redo_stations = check_stations_services.loc[check_stations_services.check < pd.Timedelta(days=-120), ['station_id', 'station_name']]
redo_stations

- locations are good
- not all stations are included!

In [ ]:
# TODO move downwards : stations_names_ids not yet defined!
stations_names_ids = pd.concat([
    df[['start_station_id', 'start_station_name']].rename(columns=lambda x: x.replace('start_','')),
    df[['end_station_id', 'end_station_name']].rename(columns=lambda x: x.replace('end_',''))
    ], axis=0).drop_duplicates()
stations_names_ids = stations_names_ids[stations_names_ids.station_id.notna() & stations_names_ids.station_name.notna()]

In [ ]:
missing_stations = pd.merge(stations_names_ids, stations_docked_bikes, on=['station_id', 'station_name'], how='outer')
missing_stations = missing_stations[missing_stations.lat_mean.isna() | missing_stations.lng_mean.isna()]
missing_stations

In [ ]:
missing_stations[missing_stations.duplicated(subset=['station_id', 'station_name'], keep=False)]

### Get station data

In [ ]:
# get all station data
start_stations = get_station_data(df, mask_start, ['ended_at', 'start_station_name', 'start_station_id', 'start_lat', 'start_lng'], dec=NDEC_LAT_LNG, drop_dups=False)
end_stations = get_station_data(df, mask_end, ['ended_at', 'end_station_name','end_station_id', 'end_lat', 'end_lng'], dec=NDEC_LAT_LNG, drop_dups=False)
stations = pd.concat(
    [start_stations, end_stations],
    axis = 0
    )

# group by 
stations = stations.groupby(['station_name', 'station_id', 'lat', 'lng']).agg({'date':['count', 'min', 'max']})
stations.columns = ['_'.join(col) for col in stations.columns]
stations.rename(columns={'date_count':'count'})
stations = stations.reset_index()

stations.head()

In [ ]:
# restrict to missing station data
mask_missing_stations = (
    (stations.station_id.isin(missing_stations.station_id) & stations.station_name.isin(missing_stations.station_name)) |
    (stations.station_id.isin(redo_stations.station_id) & stations.station_name.isin(redo_stations.station_name))
    )
print(mask_missing_stations.sum())
stations_calc = stations[mask_missing_stations]

In [ ]:
stations_calc.nunique()

In [ ]:
# plot sample on map
n_ids = 10
n_stations_per_id = 1000

# ids = station_ids.sample(n_ids)
ids = list(stations.station_id.value_counts().sort_values(ascending=False).head(n_ids).index)

stations_samples = []
for i in ids:
    stations_sample = stations[stations.station_id == i]
    if stations_sample.shape[0] > n_stations_per_id:
        stations_sample = stations_sample.sample(n_stations_per_id)
    stations_samples.append(stations_sample)
stations_samples = pd.concat(stations_samples, axis=0)
stations_samples.head(100)


In [ ]:
# scatter plot
sns.scatterplot(stations_samples, x='lat', y='lng', hue=stations_samples.station_id.astype(str))

In [ ]:
# plot on map


### Clustering of Station Data

In [ ]:
# # TODO needs some data cleaning first!
# nr_stations = len(set(zip(stations['station_id'], stations['station_name'])))

# from sklearn.neighbors import NearestNeighbors

# # write a cython function?
# # use haversine instead, it's a good approximation
# # geodesic_m = lambda x, y:  geodesic(x, y).m

# X = np.radians(np.array(list(zip(stations.lat, stations.lng))))

# nbrs = NearestNeighbors(n_neighbors=nr_stations, metric='haversine', algorithm='ball_tree', n_jobs=-1).fit(X)
# distances, indcs = nbrs.kneighbors(X)
# distances *= EARTH_RADIUS_M

In [ ]:
# stations_moved = np.apply_along_axis(lambda d: (d > 10).any(), axis=0, arr=distances)
# print(stations_moved.shape)
# stations_moved.sum()

### Remove Outliers

In [ ]:
# test data for functions below
# ids = np.random.choice(stations.station_id.unique(), size=10, replace=False)
# stations_test_set = stations[stations.station_id.apply(lambda id: id in ids)]

In [ ]:
from scipy.spatial.distance import cdist
from sklearn.neighbors import LocalOutlierFactor
import math

def dist_haversine(u, v):
    # Differences in coordinates
    dlat = v[0] - u[0]
    dlon = v[1] - u[1]

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(u[0]) * math.cos(v[0]) * math.sin(dlon / 2)**2
    return 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

def remove_outliers_lof(group, lat_lng_std_tol=1-6, dist_tol=1):
    # data prep
    v = np.array(group[['lat', 'lng']]).astype(np.float32)
    # lat_mean, lat_std = np.mean(group[['lat']]), np.std(group[['lat']])
    # lng_mean, lng_std = np.mean(group[['lng']]), np.std(group[['lng']])
    # if (lat_std<tol) & (lng_std<tol):
    #     return group
    if len(v) < 5:
        return group
    v = np.radians(v)
    u = v
    # calc distances
    distances = cdist(u, v, metric=dist_haversine)
    # apply dist tolerance
    distances[distances * EARTH_RADIUS_M < dist_tol] = 0.
    # find outliers
    lof = LocalOutlierFactor(metric='precomputed', n_jobs=-1)
    pred = lof.fit_predict(distances)
    # mark outliers
    return group.iloc[np.where(pred>0)]

In [ ]:
# try:
#     stations_wo_outliers = pd.DataFrame()
#     stations_wo_outliers = pd.read_parquet(config['processed_data_paths']['stations_wo_outliers'])
# except:
#     stations_wo_outliers = stations.groupby(['station_id', 'station_name']).apply(
#         remove_outliers_lof, lat_lng_std_tol=1e-4, dist_tol=1)

In [ ]:
stations_calc[['station_id']].value_counts()

In [ ]:
pd.DataFrame(stations_calc.groupby(['station_id', 'station_name']))

In [ ]:
stations_wo_outliers = stations_calc.groupby(['station_id', 'station_name']).apply(
    remove_outliers_lof, 
    lat_lng_std_tol=1e-4, 
    dist_tol=1
    )

In [ ]:
print(stations_calc.shape)
print(stations_wo_outliers.shape)

In [ ]:
stations_wo_outliers.describe()

In [ ]:
stations.describe()

In [ ]:
# TODO stations_wo_outliers has station_id, station_name in index and in columns - fix this
stations_wo_outliers = stations_wo_outliers.reset_index(drop=True)

### Aggregate

In [ ]:
stations_agg = stations_wo_outliers.groupby(['station_id', 'station_name']).agg({
    'date_min' : 'min',
    'date_max' : 'max',
    'lat' : ['median', 'mean', 'std', 'max', 'min'],
    'lng' : ['median', 'mean', 'std', 'max', 'min']
})
stations_agg.columns = ['_'.join(col) for col in stations_agg.columns]
stations_agg = stations_agg.reset_index()
stations_agg = stations_agg.rename(columns={'date_min_min':'start_service', 'date_max_max':'end_service'})
stations_agg

In [ ]:
# stations_agg = stations_wo_outliers.groupby(['station_id', 'station_name']).agg({
#     'date_min' : 'min',
#     'date_max' : 'max',
#     'lat' : ['mean', 'std', 'max', 'min'],
#     'lng' : ['mean', 'std', 'max', 'min']
# })

# stations_agg


### Analyse station table

In [ ]:
stations_agg.describe()

In [ ]:
# Where does the lat, lng differ strongly?
# stations_agg[(stations_agg[('lat','std')]>1e-3) | (stations_agg[('lng','std')]>1e-3)].sort_values(by=('lat','std'), ascending=False)
stations_agg[(stations_agg[('lat_std')]>1e-3) | (stations_agg[('lng_std')]>1e-3)].sort_values(by=('lat_std'), ascending=False)

In [ ]:
# calculate diagonals of station bounding boxes
# stations bounding box:
def geodesic_m(lat1, lng1, lat2, lng2):
    return [geodesic(gPoint(x1, y1),gPoint(x2, y2)).m for x1, y1, x2, y2 in zip(lat1, lng1, lat2, lng2)]

# stations_agg['diagonal_bbox'] = geodesic_m(stations_agg[('lat', 'min')], stations_agg[('lng', 'min')], stations_agg[('lat', 'max')], stations_agg[('lng', 'max')])
stations_agg['diagonal_bbox'] = geodesic_m(stations_agg[('lat_min')], stations_agg[('lng_min')], stations_agg[('lat_max')], stations_agg[('lng_max')])
stations_agg.sort_values(by='diagonal_bbox', ascending=False)

* bounding boxes are up to 20km large!
* there are large outliers in the feckin data

### Build stations table

In [ ]:
stations_docked_bikes.columns

In [ ]:
stations_agg.columns

In [ ]:
stations_docked_bikes.shape

In [ ]:
stations_agg.shape

In [ ]:
station_cols = ['station_name', 'station_id', 'lat_median','lng_median']
stations = pd.concat([stations_docked_bikes[station_cols], stations_agg[station_cols]], axis=0)
stations.head()

In [ ]:
stations_redo_1 = stations[stations.duplicated(['station_name', 'station_id'], keep='first')].set_index(['station_id', 'station_name'])
stations_redo_2 = stations[stations.duplicated(['station_name', 'station_id'], keep='last')].set_index(['station_id', 'station_name'])

diff = pd.DataFrame(data=[
    abs(stations_redo_1.lat_median - stations_redo_2.lat_median),
    abs(stations_redo_1.lng_median - stations_redo_2.lng_median) 
]).T

TOL = 1e-3
diff[(diff.lat_median>TOL) | (diff.lng_median>TOL)].empty

In [ ]:
stations = stations.drop_duplicates(subset=['station_id', 'station_name'], keep='first')

In [ ]:
stations.info()

In [ ]:
stations_ = stations.copy()

### Merge Meta Data: Count, Service Dates

- Count: How often was the feckin station targeted in the whole period?
- Service Dates: When was the first and last service date?

In [ ]:
# get all station data
start_stations = get_station_data(df, mask_start, ['ended_at', 'start_station_name', 'start_station_id'], dec=NDEC_LAT_LNG, drop_dups=False)
end_stations = get_station_data(df, mask_end, ['ended_at', 'end_station_name','end_station_id'], dec=NDEC_LAT_LNG, drop_dups=False)
stations_service_dates = pd.concat(
    [start_stations, end_stations],
    axis = 0
    )

# group by 
stations_service_dates = stations_service_dates.groupby(['station_name', 'station_id']).agg({'date':['count', 'min', 'max']})
stations_service_dates.columns = ['_'.join(col) for col in stations_service_dates.columns]
stations_service_dates = stations_service_dates.rename(columns={'date_count':'total_count_start_end'})
stations_service_dates = stations_service_dates.rename(columns={'date_min' : 'date_start', 'date_max' : 'date_end'})
stations_service_dates = stations_service_dates.reset_index()


In [ ]:
stations_service_dates.shape

In [ ]:
# stations_service_dates.loc[stations_service_dates.date_end>=pd.Timestamp('2023.12.31').date(), 'date_end'] = np.nan
stations_service_dates.sort_values(by='total_count_start_end', ascending=False)

In [ ]:
stations = pd.merge(stations, stations_service_dates, on=['station_id', 'station_name'], how='left')
stations.index.name = 'station_id'
stations = stations.rename(columns={'station_id':'station_id_old'})
stations.index = stations.index.astype('uint32[pyarrow]')  
stations

### Add new station_id column to data frame

In [ ]:
# merge new station index to dataframe
df = pd.merge(df,
         left_on=['start_station_id', 'start_station_name'],
         right=stations[['station_id_old', 'station_name']].reset_index().set_index(['station_id_old', 'station_name']),
         right_index=True,
         how='left').drop(columns=['start_station_id', 'start_station_name'])
df = df.rename(columns={'start_station_id':'start_station_id_old'})
df = df.rename(columns={'station_id':'start_station_id'})

df = pd.merge(df,
         left_on=['end_station_id', 'end_station_name'],
         right=stations[['station_id_old', 'station_name']].reset_index().set_index(['station_id_old', 'station_name']),
         right_index=True,
         how='left').drop(columns=['end_station_id', 'end_station_name'])
df = df.rename(columns={'end_station_id':'end_station_id_old'})
df = df.rename(columns={'station_id':'end_station_id'})

df.head()

## Bike Types

### Docked Bikes

In [ ]:
# get docked bikes
docked_bikes = df.loc[df.rideable_type == 'docked_bike']
print(docked_bikes.shape)
docked_bikes.head()

In [ ]:
docked_bikes.describe()

In [ ]:
# check if docked bikes are ever put outside a station
print(docked_bikes.start_station_id.isna().sum())
print(docked_bikes.end_station_id.isna().sum())

In [ ]:
df.rideable_type.value_counts()

docked_bikes can only be ridden from station to station -> classic bikes

In [ ]:
df[df.rideable_type == 'docked_bikes'].rideable_type = 'classic_bike'

### E-Bikes

In [ ]:
df_electric_bikes = df.loc[df.rideable_type == 'electric_bike']
df_electric_bikes.describe()

In [ ]:
print(df_electric_bikes.start_station_id.isna().sum(), df_electric_bikes.shape[0])

# Preprocess Data

## Create Graph Data

# Export

In [ ]:
%store df stations

In [ ]:
df.to_parquet(config['processed_data_paths']['rides'])
stations.to_parquet(config['processed_data_paths']['stations'])

# Conclusions from Data Cleaning

- Bike location measurements has to be improved: They spread far around a station.